<a href="https://colab.research.google.com/github/nuot/AL-Clean/blob/main/AssistantListing_ObliClean_InProgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assistant Listing Obligation data cleaning project In Progress** 
<br>

Author: Nuo Tian



**Last Updated: 03/01/2021**

In [3]:
import pandas as pd
import re
data = pd.read_csv('AssistanceListings.csv')

In [4]:
obligations = data['Obligations (122)']

In [5]:
types = []
for row in obligations:
  a = re.findall(r'\((.*?)\) FY',str(row))
  if a:
    types.append(a)
  else:
    types.append('None')


In [6]:
types_dedup = []
for type1 in types:
    type1 = list(dict.fromkeys(type1))
    types_dedup.append(type1)

In [7]:
data['Obligation Types'] = types_dedup

In [ ]:
data.to_csv('AssistanceListingInProgress.csv')

from google.colab import files
files.download('AssistanceListingInProgress.csv')

**Checking Point** 
<br>
Done with obligation sepration
<br>
Next working on seprating the $amount for each obligation type

In [17]:
obli = data['Obligations (122)'].str.split(';', expand=True)

In [23]:
FY_19 = []
FY_20 = [] 
for row in obli[0]:
  a = re.findall(r'FY [0-9][0-9] est \$[^\]]+[.%]?',str(row))
  FY_19.append(a)

for row in obli[1]:
  b = re.findall(r'FY 20\$[^\]]+[.%]?',str(row))
  FY_20.append(b)


In [223]:
df = pd.DataFrame()

i = 0
for i in range(len(obli.columns)):
  val = []
  for row in obli[i]:
    a = re.findall(r'FY [0-9][0-9]\$[^\]]+[.%]?',str(row))
    b = re.findall(r'FY [0-9][0-9] est \$[^\]]+[.%]?',str(row))
    if a:
      val.append(a)
    elif b:
      val.append(b)
    else:
      val.append('')
  df.insert(i, i, val, True) 


In [224]:
df = df.applymap(str)

In [225]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
0,"['FY 19$4,704,045.00']","['FY 20 est $6,617,316.00']","['FY 21 est $7,000,000.00']","['FY 18$6,271,318.00']","['FY 17$45,381,885.00']","['FY 16$45,381,885.00']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,"['FY 19$288,228,684.00']","['FY 20 est $298,125,341.00']","['FY 21 est $299,631,935.00']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,"['FY 19$8,832,031.00']","['FY 20 est $9,008,723.00']","['FY 21 est $9,459,159.00']","['FY 18$7,905,018.00']","['FY 17$8,583,390.00']","['FY 16$6,649,664.00']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,"['FY 19$12,963,091.00']","['FY 20 est $21,375,161.00']","['FY 21 est $7,092,041.00']","['FY 18$6,586,698.00']","['FY 17$6,794,055.00']","['FY 16$30,472,730.00']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,"['FY 19$3,205,858,000.00']","['FY 20 est $3,159,314,000.00']","['FY 21 est $3,888,218,000.00']","['FY 18$3,145,610,000.00']","['FY 17$6,914,144,153.00']","['FY 16$6,560,230,313.00']","['FY 19$33,451,000.00']","['FY 20 est $65,467,000.00']","['FY 21 est $159,554,000.00']","['FY 18$72,000.00']","['FY 17$46,564,581.00']","['FY 16$212,967,255.00']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2270,"['FY 19$1,000,000,000.00']","['FY 20 est $1,000,000,000.00']","['FY 18$1,060,362,596.00']","['FY 21 est $1,000,000,000.00']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2271,"['FY 19$15,000,000.00']","['FY 20 est $15,000,000.00']","['FY 18$14,854,231.00']","['FY 21 est $15,000,000.00']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2272,['FY 19$0.00'],['FY 20 est $0.00'],['FY 18$0.00'],['FY 21 est $0.00'],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2273,"['FY 19$17,095,000.00']","['FY 20 est $17,200,000.00']","['FY 18$12,828,336.00']","['FY 21 est $3,500,000.00']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [226]:
df['combined'] = df.apply(lambda row: ''.join(row),axis = 1)

In [227]:
df['combined']

0       ['FY 19$4,704,045.00']['FY 20 est $6,617,316.0...
1       ['FY 19$288,228,684.00']['FY 20 est $298,125,3...
2       ['FY 19$8,832,031.00']['FY 20 est $9,008,723.0...
3       ['FY 19$12,963,091.00']['FY 20 est $21,375,161...
4       ['FY 19$3,205,858,000.00']['FY 20 est $3,159,3...
                              ...                        
2270    ['FY 19$1,000,000,000.00']['FY 20 est $1,000,0...
2271    ['FY 19$15,000,000.00']['FY 20 est $15,000,000...
2272    ['FY 19$0.00']['FY 20 est $0.00']['FY 18$0.00'...
2273    ['FY 19$17,095,000.00']['FY 20 est $17,200,000...
2274    ['FY 19$30,000,000.00']['FY 20 est $31,000,000...
Name: combined, Length: 2275, dtype: object

In [228]:
df['combined'][10]

"['FY 19$22,152,000.00']['FY 20 est $16,088,000.00']['FY 21 est $0.00']['FY 18$122,328,000.00']['FY 17$88,663,000.00']['FY 16$39,735,000.00']['FY 19$6,190,000.00']['FY 20 est $4,698,000.00']['FY 21 est $0.00']['FY 18$30,646,000.00']['FY 17$38,616,000.00']['FY 16$14,591,000.00']"

In [229]:
newdf = df['combined'].str.split('FY 19', expand=True)

In [230]:
newdf['type1'] = 'FY 19' + newdf[1].astype(str)

In [235]:
len(newdf[2])

for i in range(len(newdf[2])):
  if newdf[2][i] != None:
    newdf['type2'][i] = 'FY 19' + newdf[2][i]
  else:
    newdf['type2'][i] = ''

In [236]:
newdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,type1,type2
0,[',"$4,704,045.00']['FY 20 est $6,617,316.00']['FY...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FY 19$4,704,045.00']['FY 20 est $6,617,316.00'...",
1,[',"$288,228,684.00']['FY 20 est $298,125,341.00']...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FY 19$288,228,684.00']['FY 20 est $298,125,341...",
2,[',"$8,832,031.00']['FY 20 est $9,008,723.00']['FY...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FY 19$8,832,031.00']['FY 20 est $9,008,723.00'...",
3,[',"$12,963,091.00']['FY 20 est $21,375,161.00']['...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FY 19$12,963,091.00']['FY 20 est $21,375,161.0...",
4,[',"$3,205,858,000.00']['FY 20 est $3,159,314,000....","$33,451,000.00']['FY 20 est $65,467,000.00']['...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FY 19$3,205,858,000.00']['FY 20 est $3,159,314...","FY 19$33,451,000.00']['FY 20 est $65,467,000.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2270,[',"$1,000,000,000.00']['FY 20 est $1,000,000,000....",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FY 19$1,000,000,000.00']['FY 20 est $1,000,000...",
2271,[',"$15,000,000.00']['FY 20 est $15,000,000.00']['...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FY 19$15,000,000.00']['FY 20 est $15,000,000.0...",
2272,[',$0.00']['FY 20 est $0.00']['FY 18$0.00']['FY 2...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,FY 19$0.00']['FY 20 est $0.00']['FY 18$0.00'][...,
2273,[',"$17,095,000.00']['FY 20 est $17,200,000.00']['...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FY 19$17,095,000.00']['FY 20 est $17,200,000.0...",
